In [1]:
from keras.models import Sequential
import keras
import pandas as pd 
import numpy as np 
import lightgbm as lgb
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from Dataloader import Dataloader
from RamdomForest import RamdomForest
from score import score
from sklearn.utils import shuffle
from IPython.display import display, HTML
from datetime import date
from keras.layers import Dense
import keras.backend as K
from sklearn import preprocessing
import category_encoders as ce
%matplotlib inline

Using TensorFlow backend.


In [2]:
df = pd.read_csv('data/train.csv')
parm = {
    'learning_rate' : 0.01,
    'n_estimators' : 1000,
    'max_depth' : [32],
    'num_leaves' : [256],
    'feature_fraction' : [0.9]
}

KeyboardInterrupt: 

In [ ]:
dataloader = Dataloader()
X_train, X_test, y_train, y_test = dataloader.prepare_train_data(df)

In [ ]:
ramdom_forest = RamdomForest()
model = ramdom_forest.train_LGBM(X_train, y_train, X_test, y_test,parm)

In [ ]:
ax = model.plot_feature_important()

In [ ]:
model.plot_loss()

In [ ]:
y, y_true = model.predict(X_train,y_train)
test_y, test_y_true = model.predict(X_test,y_test)
nn_model = Sequential()
nn_model.add(Dense(units=1, input_dim=1))
nn_model.compile(loss=keras.losses.MSE,
              optimizer=keras.optimizers.SGD(lr=0.01))
history = nn_model.fit(y, y_true, epochs=20, batch_size=32,validation_data=(test_y, test_y_true))

y = nn_model.predict(test_y, batch_size=128)
y_true = test_y_true

In [ ]:
hit_rate,point = score(y,y_true)
print(hit_rate)

In [ ]:
test_df = pd.read_csv("data/test.csv")
build_id = pd.read_csv("data/submit_test.csv")['building_id']
test_df = dataloader.prepare_test_data(test_df)
test_y, _= model.predict(test_df,y_test)
submit_df = pd.DataFrame(data={
    'building_id' : build_id,
    'total_price' : test_y
})
today = str(date.today())
submit_df.to_csv(f'submits/{today}_lightgbm.csv',index = False)